In [13]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist_m as mnistm
from data import mnist
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from data.tasks import load_task
from experiments.training import *
from experiments.SL_bound import *
from experiments.DA_bound import *
from bounds.bounds import *
from util.kl import *
from util.misc import *
from util.batch import *
from results.plotting import *

# Hyper-parameters
task = 2
seed = 69105
n_classifiers = 10
delta=0.05 ## what would this be?   
binary=True
bound='germain'
epsilons=[0.03]#,0.001]
alphas=[0.3]#, 0.1,0.3,0.5]#0,0.3]
sigmas=[[3,1],[3,2],[3,3]]

project_folder = "/cephyr/users/adambre/Alvis/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Alvis params

In [4]:
project = 'SNIC2021-7-83'
username = 'adambre'
job = 'batch_bound_single.sbatch'

In [47]:
!squeue -u $username

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
            105740     alvis     bash  adambre  R 4-11:55:30      1 alvis1-02 
            106790     alvis     bash  adambre  R   23:29:27      1 alvis1-03 


In [48]:
!cat $job

#!/usr/bin/bash

#SBATCH -t 1-00:00:00
#SBATCH -N 1 --gpus-per-node=T4:1
#SBATCH -p alvis 

if [ -z "$task" ]
then
    task=2
fi
if [ -z "$seed" ]
then 
    seed=69105
fi
if [ -z "$alpha" ]
then 
    alpha=0.0
fi
if [ -z "$sigma" ]
then 
    sigma='3,3'
fi
if [ -z "$epsilon" ]
then 
    epsilon=0.01
fi
if [ -z "$delta" ]
then 
    delta=0.05
fi
if [ -z "$binary" ]
then 
    binary=0
fi
if [ -z "$nclassifiers" ]
then 
    nclassifiers=2
fi
if [ -z "$bound" ]
then 
    bound='germain'
fi
if [ -z "$prior" ]
then 
    prior=''
fi
if [ -z "$posterior" ]
then 
    posterior=''
fi

. load_modules.sh

python batch_bound_single.py -t $task -r $seed -a $alpha -s $sigma -e $epsilon -d $delta -b $binary -n $nclassifiers -B $bound -p $prior -P $posterior

In [14]:
print('Iterating over experiments...\n')

fids = []

os.makedirs('logs', exist_ok=True)


np.random.seed(seed)
for s in range(10):
    seed+=1
    print(seed)
    for alpha in alphas:

        print("alpha:"+str(alpha))

        for epsilon in epsilons:
            print("  epsilon:"+str(epsilon))
            for sigma in sigmas:    
                print("    sigma:"+str(sigma))
                arg_list = get_job_args(task, bound=bound, alpha=alpha, sigma=sigma,
                                        epsilon=epsilon, binary=binary, n_classifiers=n_classifiers)
                for a in arg_list[5:]:   

                    ckpt = os.path.splitext(os.path.basename(a['posterior_path']))[0]
                    fid = './logs/batch_t-%d_r-%d_a-%.4f_s-%d%d_e-%.4f_d-%.4f_b-%d_n-%d_B-%s_c-%s' % \
                        (task, seed, alpha, sigma[0], sigma[1], epsilon, delta, binary, n_classifiers, bound, ckpt)

                    sigstr = '"%d.%d"' % (sigma[0], sigma[1])
                    exp = 'task=%d,seed=%d,alpha=%.4f,sigma="%s",epsilon=%.4f,delta=%.4f,'% (task, seed, alpha, sigstr, epsilon, delta)\
                            +'binary=%d,nclassifiers=%d,bound=%s,prior=%s,posterior=%s' % (binary, n_classifiers, bound, a['prior_path'], a['posterior_path'])

                    prior_path = a['prior_path']
                    output = !sbatch -o "$fid"-%j.out -e "$fid"-%j.err -A $project --export="$exp" $job

                    jobid = int(output[0].split(' ')[-1])
                    fid = fid+'-%s' % jobid

                    fids.append(fid)

Iterating over experiments...

69106
alpha:0.3
  epsilon:0.03
    sigma:[3, 1]
    sigma:[3, 2]
    sigma:[3, 3]
69107
alpha:0.3
  epsilon:0.03
    sigma:[3, 1]
    sigma:[3, 2]
    sigma:[3, 3]
69108
alpha:0.3
  epsilon:0.03
    sigma:[3, 1]
    sigma:[3, 2]
    sigma:[3, 3]
69109
alpha:0.3
  epsilon:0.03
    sigma:[3, 1]
    sigma:[3, 2]
    sigma:[3, 3]
69110
alpha:0.3
  epsilon:0.03
    sigma:[3, 1]
    sigma:[3, 2]
    sigma:[3, 3]
69111
alpha:0.3
  epsilon:0.03
    sigma:[3, 1]
    sigma:[3, 2]
    sigma:[3, 3]
69112
alpha:0.3
  epsilon:0.03
    sigma:[3, 1]
    sigma:[3, 2]
    sigma:[3, 3]
69113
alpha:0.3
  epsilon:0.03
    sigma:[3, 1]
    sigma:[3, 2]
    sigma:[3, 3]
69114
alpha:0.3
  epsilon:0.03
    sigma:[3, 1]
    sigma:[3, 2]
    sigma:[3, 3]
69115
alpha:0.3
  epsilon:0.03
    sigma:[3, 1]
    sigma:[3, 2]
    sigma:[3, 3]


In [6]:
print(fids)

['./logs/batch_t-2_r-69105_a-0.0000_s-31_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_275-120544', './logs/batch_t-2_r-69105_a-0.0000_s-31_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_330-120545', './logs/batch_t-2_r-69105_a-0.0000_s-31_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_385-120546', './logs/batch_t-2_r-69105_a-0.0000_s-31_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_440-120547', './logs/batch_t-2_r-69105_a-0.0000_s-31_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_495-120548', './logs/batch_t-2_r-69105_a-0.0000_s-31_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-2_1-120549', './logs/batch_t-2_r-69105_a-0.0000_s-31_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-2_2-120550', './logs/batch_t-2_r-69105_a-0.0000_s-31_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-2_3-120551', './logs/batch_t-2_r-69105_a-0.0000_s-31_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-2_4-120552', './logs/batch_t-2_r-69105_a-0.0000_s-32_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_275-120553', './logs/batch_t-2_r-69105_a-0.0000_s-32_e-0.0300_d-0.0500_b-1_n-10_

In [20]:
!squeue -u $username

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
            120538     alvis     bash  adambre  R    1:42:33      1 alvis1-05 


In [28]:
for fid in fids: 
    print('------')
    print(fid)
    !tail -n 5 "$fid".out
    !tail -n 5 "$fid".err
    print(' \n\n')

------
./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_150-107647
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_150-107647.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_150-107647.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_180-107648
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_180-107648.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_180-107648.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0300_d-0.0500_b-1_n-10_B-germain_c-1_210-107649
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0300_d-0.0500_b-1_n-

 


------
./logs/batch_t-1_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_270-107673
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=107673&from=1633362873000
2021-10-04 17:54:36.625352: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
 


------
./logs/batch_t-1_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-2_1-107674
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=107674&from=1633362873000
2021-10-04 17:54:36.662321: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
 


------
./logs/batch_t-1_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-2_2-107675
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=107675&from=1633362874000
2021-10-04 17:54:36.875144: I tensorf

tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_11-107694.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_12-107695
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_12-107695.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_12-107695.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_13-107696
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_13-107696.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_13-107696.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-691

 


------
./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_32-107715
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_32-107715.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_32-107715.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_33-107716
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_33-107716.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_33-107716.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_34-107717
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-32_e-0.0010_d-0.0500_b-1_n-10_

tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_5-107735.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.1000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_6-107736
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_6-107736.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_6-107736.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.1000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_7-107737
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_7-107737.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.1000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_7-107737.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.

...

 


------
./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_180-107920
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_180-107920.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_180-107920.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_210-107921
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_210-107921.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_210-107921.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_240-107922
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-

tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_17-107940.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_18-107941
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_18-107941.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_18-107941.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_19-107942
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_19-107942.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-32_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_19-107942.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-691

 


------
./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_150-107961
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_150-107961.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_150-107961.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_180-107962
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_180-107962.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_180-107962.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-1_210-107963
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-

tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_16-107981.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_17-107982
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_17-107982.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_17-107982.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_18-107983
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_18-107983.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_18-107983.err’ for reading: No such file or directory
 


------
./logs/batch_t-1_r-691

 


------
./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_37-108002
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_37-108002.out’ for reading: No such file or directory
tail: cannot open ‘./logs/batch_t-1_r-69105_a-0.5000_s-33_e-0.0010_d-0.0500_b-1_n-10_B-germain_c-2_37-108002.err’ for reading: No such file or directory
 


